<a href="https://colab.research.google.com/github/RanaTayyab/100-Days-Of-ML-Code/blob/master/CNN_TF_Raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls

sample_data


In [5]:
!apt-get install p7zip-full


Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.


In [6]:
!p7zip -d file_name.tar.7z


/usr/bin/p7zip: cannot read file_name.tar.7z


In [7]:
!p7zip -d cow_data.7z



7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 9032535 bytes (8821 KiB)

Extracting archive: cow_data.7z
--
Path = cow_data.7z
Type = 7z
Physical Size = 9032535
Headers Size = 171854
Method = LZMA:25
Solid = +
Blocks = 1

  0%     11% 691 - data/others/10561.jpg                                 11% 1305 - data/others/11063.jpg                                  11% 1730 - data/others/11410.jpg                                  11% 2078 - data/others/11698.jpg                                  11% 2

In [9]:
"""
CowTrainer class
@author Glenn De Backer <glenn at simplicity dot be>
@license GPLv3
"""
import os
import glob

import numpy as np

from skimage import io
from sklearn.model_selection import train_test_split

from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import to_categorical

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.metrics import Accuracy
from tflearn import DNN

class CowTrainer(object):
    """ Cow trainer """
    def __init__(self, path_cow_images, path_non_cow_images):
        """ default constructor """
        # path information
        self.path_cow_images = path_cow_images
        self.path_non_cow_images = path_non_cow_images

        # images information
        self.image_size = 32 # 32x32
        self.list_cow_files = []
        self.list_noncow_files = []
        self.total_images_count = 0

        # tensorflow dataset
        self.tf_data_counter = 0
        self.tf_image_data = None
        self.tf_image_labels = None
        self.tf_x = None
        self.tf_x_test = None
        self.tf_y = None
        self.tf_y_test = None

        # tensorflow network variables
        self.tf_img_prep = None
        self.tf_img_aug = None
        self.tf_network = None

    def init_np_variables(self):
        """ Initialize NP datastructures """
        self.tf_image_data = np.zeros((self.total_images_count, self.image_size,
                                       self.image_size, 3), dtype='float64')

        self.tf_image_labels = np.zeros(self.total_images_count)

    def train(self):
        """ Start training """
        # 1: build a list of image filenames
        self.build_image_filenames_list()

        # 2: use list information to init our numpy variables
        self.init_np_variables()

        # 3: Add images to our Tensorflow dataset
        self.add_tf_dataset(self.list_cow_files, 0)
        self.add_tf_dataset(self.list_noncow_files, 1)

        # 4: Process TF dataset
        self.process_tf_dataset()

        # 5: Setup image preprocessing
        self.setup_image_preprocessing()

        # 6: Setup network structure
        self.setup_nn_network()

        # 7: Train our deep neural network
        tf_model = DNN(self.tf_network, tensorboard_verbose=3,
                       checkpoint_path='model_cows.tfl.ckpt')

        tf_model.fit(self.tf_x, self.tf_y, n_epoch=100, shuffle=True,
                     validation_set=(self.tf_x_test, self.tf_y_test),
                     show_metric=True, batch_size=96,
                     snapshot_epoch=True,
                     run_id='model_cows')

        # 8: Save model
        tf_model.save('model_cows.tflearn')

    def build_image_filenames_list(self):
        """ Get list of filenames for cows and non cows """
        self.list_cow_files = sorted(glob.glob(self.path_cow_images))
        self.list_noncow_files = sorted(glob.glob(self.path_non_cow_images))
        self.total_images_count = len(self.list_cow_files) + len(self.list_noncow_files)

    def add_tf_dataset(self, list_images, label):
        """ Add tensorflow data we will pass to our network """
        # process list of images
        for image_file in list_images:
            try:
                # read, store image and label
                img = io.imread(image_file)
                self.tf_image_data[self.tf_data_counter] = np.array(img)
                self.tf_image_labels[self.tf_data_counter] = label

                # increase counter
                self.tf_data_counter += 1
            except:
                # on error continue to the next image
                continue

    def process_tf_dataset(self):
        """ Process our TF dataset """
        # split our tf set in a test and training part
        self.tf_x, self.tf_x_test, self.tf_y, self.tf_y_test = train_test_split(
            self.tf_image_data, self.tf_image_labels, test_size=0.1, random_state=42)

        # encode our labels
        self.tf_y = to_categorical(self.tf_y, 2)
        self.tf_y_test = to_categorical(self.tf_y_test, 2)

    def setup_image_preprocessing(self):
        """ Setup image preprocessing """
        # normalization of images
        self.tf_img_prep = ImagePreprocessing()
        self.tf_img_prep.add_featurewise_zero_center()
        self.tf_img_prep.add_featurewise_stdnorm()

        # Randomly create extra image data by rotating and flipping images
        self.tf_img_aug = ImageAugmentation()
        self.tf_img_aug.add_random_flip_leftright()
        self.tf_img_aug.add_random_rotation(max_angle=30.)

    def setup_nn_network(self):
        """ Setup neural network structure """

        # our input is an image of 32 pixels high and wide with 3 channels (RGB)
        # we will also preprocess and create synthetic images
        self.tf_network = input_data(shape=[None, self.image_size, self.image_size, 3],
                                     data_preprocessing=self.tf_img_prep,
                                     data_augmentation=self.tf_img_aug)

        # layer 1: convolution layer with 32 filters (each being 3x3x3)
        layer_conv_1 = conv_2d(self.tf_network, 32, 3, activation='relu', name='conv_1')

        # layer 2: max pooling layer
        self.tf_network = max_pool_2d(layer_conv_1, 2)

        # layer 3: convolution layer with 64 filters
        layer_conv_2 = conv_2d(self.tf_network, 64, 3, activation='relu', name='conv_2')

        # layer 4: Another convolution layer with 64 filters
        layer_conv_3 = conv_2d(layer_conv_2, 64, 3, activation='relu', name='conv_3')

        # layer 5: Max pooling layer
        self.tf_network = max_pool_2d(layer_conv_3, 2)

        # layer 6: Fully connected 512 node layer
        self.tf_network = fully_connected(self.tf_network, 512, activation='relu')

        # layer 7: Dropout layer (removes neurons randomly to combat overfitting)
        self.tf_network = dropout(self.tf_network, 0.5)

        # layer 8: Fully connected layer with two outputs (cow or non cow class)
        self.tf_network = fully_connected(self.tf_network, 2, activation='softmax')

        # define how we will be training our network
        accuracy = Accuracy(name="Accuracy")
        self.tf_network = regression(self.tf_network, optimizer='adam',
                                     loss='categorical_crossentropy',
                                     learning_rate=0.0005, metric=accuracy)

# path where we can find our images
PATH_COW_FILES = os.path.join('data/cows', '*.jpg')
PATH_NONCOW_FILES = os.path.join('data/others', '*.jpg')

# start training
COWTRAINER = CowTrainer(PATH_COW_FILES, PATH_NONCOW_FILES)
COWTRAINER.train()

Training Step: 30199  | total loss: 0.01387 | time: 107.191s
| Adam | epoch: 100 | loss: 0.01387 - Accuracy: 0.9936 -- iter: 28896/28928
Training Step: 30200  | total loss: 0.01261 | time: 108.568s
| Adam | epoch: 100 | loss: 0.01261 - Accuracy: 0.9942 | val_loss: 0.09487 - val_acc: 0.9823 -- iter: 28928/28928
--
INFO:tensorflow:/content/model_cows.tfl.ckpt-30200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/content/model_cows.tflearn is not in all_model_checkpoint_paths. Manually adding it.
